<a href="https://colab.research.google.com/github/OswaldoMoper/Mathematical_Data_Analysis/blob/master/Data%20Analyst%20Project/Project_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [335]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
from datetime import datetime, date
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [336]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/ My \Drive/Diplomado/F1-1950-2023

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/ My Drive/Diplomado/F1-1950-2023


#General Functions

In [337]:
def miliseconds_to_hours(miliseconds):
    seconds = (miliseconds / 1000) % 60
    minutes = (miliseconds / (1000 * 60)) % 60
    hours = (miliseconds / (1000 * 60 * 60)) % 24
    return f"{int(hours)}:{int(minutes)}:{int(seconds)}"

In [338]:
def hours_to_miliseconds(time):
  time_dt = datetime.strptime(time, "%H:%M:%S.%f")
  miliseconds = time_dt.microsecond // 1000
  return miliseconds

In [339]:
Finished_type = CategoricalDtype(categories = ['Finished', '+1 Lap' ,'+2 Laps', '+3 Laps', '+4 Laps', '+5 Laps', '+6 Laps', '+7 Lap' ,'+8 Laps', '+9 Laps', '+10 Laps', '+11 Lap' ,'+12 Laps', '+13 Laps', '+14 Laps', '+15 Laps' ], ordered = True )
Grid_type = CategoricalDtype(categories = ['1' ,'2', '3', '4', '5', '6', '7' ,'8', '9', '10', '11' ,'12', '13', '14', '15', '16', '17' ,'18', '19', '20', '21', '22', '23', '24', '0' ], ordered = True )

In [340]:
#Función para dar formato al titulo
def format_title(title, subtitle=None, subtitle_font_size=14):
        title = f'<b>{title}</b>'
        if not subtitle:
            return title
        subtitle = f'<span style="font-size: {subtitle_font_size}px;">{subtitle}</span>'
        return f'{title}<br>{subtitle}'

#Data

##Drivers

In [341]:
driver_standings = pd.read_csv('driver_standings.csv').drop( columns = ['positionText', 'driverStandingsId'] ).rename( columns= { 'position':'standing_pos', 'points':'standing_score' } )
driver_standings

,raceId,driverId,standing_score,standing_pos,wins
0,18,1,10.0,1,1
1,18,2,8.0,2,0
2,18,3,6.0,3,0
3,18,4,5.0,4,0
4,18,5,4.0,5,0
...,...,...,...,...,...
34119,1110,846,69.0,8,0
34120,1110,839,35.0,10,0
34121,1110,844,99.0,5,0
34122,1110,857,34.0,11,0


In [342]:
drivers = pd.read_csv('drivers.csv').drop( columns = [ 'url', 'driverRef' ] ).rename( columns = { 'number': 'dorsal' } )
drivers

,driverId,dorsal,code,forename,surname,dob,nationality
0,1,44,HAM,Lewis,Hamilton,1985-01-07,British
1,2,\N,HEI,Nick,Heidfeld,1977-05-10,German
2,3,6,ROS,Nico,Rosberg,1985-06-27,German
3,4,14,ALO,Fernando,Alonso,1981-07-29,Spanish
4,5,\N,KOV,Heikki,Kovalainen,1981-10-19,Finnish
...,...,...,...,...,...,...,...
852,854,47,MSC,Mick,Schumacher,1999-03-22,German
853,855,24,ZHO,Guanyu,Zhou,1999-05-30,Chinese
854,856,21,DEV,Nyck,de Vries,1995-02-06,Dutch
855,857,81,PIA,Oscar,Piastri,2001-04-06,Australian


##Races

In [343]:
circuits = pd.read_csv('circuits.csv').rename( columns = { 'name': 'circuitName' } ).drop( columns = [ 'url', 'circuitRef' ] )
circuits

,circuitId,circuitName,location,country,lat,lng,alt
0,1,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10
1,2,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,18
2,3,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,7
3,4,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57000,2.26111,109
4,5,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,130
...,...,...,...,...,...,...,...
72,75,Autódromo Internacional do Algarve,Portimão,Portugal,37.22700,-8.62670,108
73,76,Autodromo Internazionale del Mugello,Mugello,Italy,43.99750,11.37190,255
74,77,Jeddah Corniche Circuit,Jeddah,Saudi Arabia,21.63190,39.10440,15
75,78,Losail International Circuit,Al Daayen,Qatar,25.49000,51.45420,\N


In [344]:
races = pd.read_csv('races.csv').rename( columns = { 'name': 'GPName' } ).drop( columns = [ 'url', 'fp1_date', 'fp1_time', 'fp2_date', 'fp2_time', 'fp3_date', 'fp3_time', 'quali_date', 'quali_time', 'sprint_date', 'sprint_time' ] )
races

,raceId,year,round,circuitId,GPName,date,time
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00
...,...,...,...,...,...,...,...
1096,1116,2023,18,69,United States Grand Prix,2023-10-22,19:00:00
1097,1117,2023,19,32,Mexico City Grand Prix,2023-10-29,20:00:00
1098,1118,2023,20,18,São Paulo Grand Prix,2023-11-05,17:00:00
1099,1119,2023,21,80,Las Vegas Grand Prix,2023-11-19,06:00:00


In [345]:
GrandPrix = races.merge( circuits, on = [ 'circuitId' ], how = 'left' )
GrandPrix

,raceId,year,round,circuitId,GPName,date,time,circuitName,location,country,lat,lng,alt
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,18
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,Shanghai International Circuit,Shanghai,China,31.33890,121.22000,5
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,7
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57000,2.26111,109
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1096,1116,2023,18,69,United States Grand Prix,2023-10-22,19:00:00,Circuit of the Americas,Austin,USA,30.13280,-97.64110,161
1097,1117,2023,19,32,Mexico City Grand Prix,2023-10-29,20:00:00,Autódromo Hermanos Rodríguez,Mexico City,Mexico,19.40420,-99.09070,2227
1098,1118,2023,20,18,São Paulo Grand Prix,2023-11-05,17:00:00,Autódromo José Carlos Pace,São Paulo,Brazil,-23.70360,-46.69970,785
1099,1119,2023,21,80,Las Vegas Grand Prix,2023-11-19,06:00:00,Las Vegas Strip Street Circuit,Las Vegas,United States,36.11470,-115.17300,\N


In [346]:
qualifying = pd.read_csv('qualifying.csv').rename( columns = { 'number':'dorsal', 'position': 'qualy_position' } )
qualifying

,qualifyId,raceId,driverId,constructorId,dorsal,qualy_position,q1,q2,q3
0,1,18,1,1,22,1,1:26.572,1:25.187,1:26.714
1,2,18,9,2,4,2,1:26.103,1:25.315,1:26.869
2,3,18,5,1,23,3,1:25.664,1:25.452,1:27.079
3,4,18,13,6,2,4,1:25.994,1:25.691,1:27.178
4,5,18,2,2,3,5,1:25.960,1:25.518,1:27.236
...,...,...,...,...,...,...,...,...,...
9810,9868,1110,848,3,23,16,2:00.314,\N,\N
9811,9869,1110,855,51,24,17,2:00.832,\N,\N
9812,9870,1110,858,3,2,18,2:01.535,\N,\N
9813,9871,1110,817,213,3,19,2:02.159,\N,\N


In [347]:
lap_times = pd.read_csv('lap_times.csv')
lap_times

,raceId,driverId,lap,position,time,milliseconds
0,841,20,1,1,1:38.109,98109
1,841,20,2,1,1:33.006,93006
2,841,20,3,1,1:32.713,92713
3,841,20,4,1,1:32.803,92803
4,841,20,5,1,1:32.342,92342
...,...,...,...,...,...,...
551737,1110,817,40,17,1:54.361,114361
551738,1110,817,41,17,1:53.367,113367
551739,1110,817,42,16,1:55.247,115247
551740,1110,817,43,16,1:52.115,112115


In [348]:
pit_stops = pd.read_csv('pit_stops.csv')
pit_stops

,raceId,driverId,stop,lap,time,duration,milliseconds
0,841,153,1,1,17:05:23,26.898,26898
1,841,30,1,1,17:05:52,25.021,25021
2,841,17,1,11,17:20:48,23.426,23426
3,841,4,1,12,17:22:34,23.251,23251
4,841,13,1,13,17:24:10,23.842,23842
...,...,...,...,...,...,...,...
10084,1110,4,2,29,15:59:01,23.798,23798
10085,1110,830,2,30,16:00:16,23.012,23012
10086,1110,848,3,33,16:07:06,23.529,23529
10087,1110,858,3,34,16:09:09,23.109,23109


In [349]:
Q = qualifying.drop( columns = [ 'qualifyId', 'constructorId', 'dorsal', 'q1', 'q2', 'q3' ])
Q

,raceId,driverId,qualy_position
0,18,1,1
1,18,9,2
2,18,5,3
3,18,13,4
4,18,2,5
...,...,...,...
9810,1110,848,16
9811,1110,855,17
9812,1110,858,18
9813,1110,817,19


##Results

In [350]:
results = pd.read_csv('results.csv').drop( columns = ['resultId', 'constructorId', 'positionText', 'positionOrder', 'points', 'time', 'fastestLap', 'rank', 'fastestLapTime' ,'fastestLapSpeed' ]  )
#results = Q.merge(result, on = ['raceId', 'driverId'], how = 'right' ).drop(columns='grid').rename(columns= {'qualy_position':'grid'} )
results

,raceId,driverId,number,grid,position,laps,milliseconds,statusId
0,18,1,22,1,1,58,5690616,1
1,18,2,3,5,2,58,5696094,1
2,18,3,7,7,3,58,5698779,1
3,18,4,5,11,4,58,5707797,1
4,18,5,23,3,5,58,5708630,1
...,...,...,...,...,...,...,...,...
26075,1110,817,3,19,16,44,5053521,1
26076,1110,858,2,18,17,44,5054926,1
26077,1110,807,27,0,18,44,5060900,1
26078,1110,832,55,4,\N,23,\N,130


In [351]:
status = pd.read_csv('status.csv')
status

,statusId,status
0,1,Finished
1,2,Disqualified
2,3,Accident
3,4,Collision
4,5,Engine
...,...,...
134,137,Damage
135,138,Debris
136,139,Illness
137,140,Undertray


In [352]:
R = results.merge(status, on = [ 'statusId' ], how = 'left' ).rename(columns = { 'position':'final_pos' } ).drop( columns = 'statusId' )
R.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26080 entries, 0 to 26079
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   raceId        26080 non-null  int64 
 1   driverId      26080 non-null  int64 
 2   number        26080 non-null  object
 3   grid          26080 non-null  int64 
 4   final_pos     26080 non-null  object
 5   laps          26080 non-null  int64 
 6   milliseconds  26080 non-null  object
 7   status        26080 non-null  object
dtypes: int64(4), object(4)
memory usage: 1.8+ MB


In [353]:
R

,raceId,driverId,number,grid,final_pos,laps,milliseconds,status
0,18,1,22,1,1,58,5690616,Finished
1,18,2,3,5,2,58,5696094,Finished
2,18,3,7,7,3,58,5698779,Finished
3,18,4,5,11,4,58,5707797,Finished
4,18,5,23,3,5,58,5708630,Finished
...,...,...,...,...,...,...,...,...
26075,1110,817,3,19,16,44,5053521,Finished
26076,1110,858,2,18,17,44,5054926,Finished
26077,1110,807,27,0,18,44,5060900,Finished
26078,1110,832,55,4,\N,23,\N,Collision damage


#Ayrton Senna

##Code

In [354]:
AS = drivers[ drivers.forename == 'Ayrton' ].reset_index().drop(columns='index')
AS_Id = AS.loc[0, 'driverId']
AS

,driverId,dorsal,code,forename,surname,dob,nationality
0,102,\N,\N,Ayrton,Senna,1960-03-21,Brazilian


In [355]:
AS_standings = driver_standings[ driver_standings.driverId == AS_Id ].merge( AS, on = 'driverId', how = 'left' ).drop( columns = ['dob', 'nationality', 'driverId', 'dorsal', 'code' ] )
AS_standings

,raceId,standing_score,standing_pos,wins,forename,surname
0,258,0.0,19,0,Ayrton,Senna
1,259,0.0,23,0,Ayrton,Senna
2,260,0.0,27,0,Ayrton,Senna
3,261,0.0,29,0,Ayrton,Senna
4,262,0.0,30,0,Ayrton,Senna
...,...,...,...,...,...,...
170,448,9.0,11,0,Ayrton,Senna
171,449,9.0,11,0,Ayrton,Senna
172,450,9.0,11,0,Ayrton,Senna
173,451,13.0,9,0,Ayrton,Senna


In [356]:
AS_standings.forename.unique()

array(['Ayrton'], dtype=object)

In [357]:
AS_result = R[ R.driverId == AS_Id ].reset_index()
AS_results = AS_result.merge(AS_standings, on = 'raceId', how = 'left' ).drop(columns = 'index')
AS_results

,raceId,driverId,number,grid,final_pos,laps,milliseconds,status,standing_score,standing_pos,wins,forename,surname
0,257,102,2,1,\N,55,\N,Spun off,0.0,13.0,0.0,Ayrton,Senna
1,258,102,2,1,\N,0,\N,Collision,0.0,19.0,0.0,Ayrton,Senna
2,259,102,2,1,\N,5,\N,Accident,0.0,23.0,0.0,Ayrton,Senna
3,273,102,8,2,2,72,6004906,Finished,6.0,2.0,0.0,Ayrton,Senna
4,274,102,8,3,1,71,6675485,Finished,16.0,1.0,1.0,Ayrton,Senna
...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,446,102,19,9,\N,4,\N,Accident,9.0,10.0,0.0,Ayrton,Senna
158,447,102,19,10,\N,35,\N,Oil pressure,9.0,10.0,0.0,Ayrton,Senna
159,448,102,19,13,\N,19,\N,Engine,9.0,11.0,0.0,Ayrton,Senna
160,450,102,19,12,\N,0,\N,Accident,9.0,11.0,0.0,Ayrton,Senna


In [358]:
AS_results.driverId.unique()

array([102])

##Results

In [359]:
AS_R = AS_results.copy()
AS_R.driverId.unique()

array([102])

In [360]:
AS_R

,raceId,driverId,number,grid,final_pos,laps,milliseconds,status,standing_score,standing_pos,wins,forename,surname
0,257,102,2,1,\N,55,\N,Spun off,0.0,13.0,0.0,Ayrton,Senna
1,258,102,2,1,\N,0,\N,Collision,0.0,19.0,0.0,Ayrton,Senna
2,259,102,2,1,\N,5,\N,Accident,0.0,23.0,0.0,Ayrton,Senna
3,273,102,8,2,2,72,6004906,Finished,6.0,2.0,0.0,Ayrton,Senna
4,274,102,8,3,1,71,6675485,Finished,16.0,1.0,1.0,Ayrton,Senna
...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,446,102,19,9,\N,4,\N,Accident,9.0,10.0,0.0,Ayrton,Senna
158,447,102,19,10,\N,35,\N,Oil pressure,9.0,10.0,0.0,Ayrton,Senna
159,448,102,19,13,\N,19,\N,Engine,9.0,11.0,0.0,Ayrton,Senna
160,450,102,19,12,\N,0,\N,Accident,9.0,11.0,0.0,Ayrton,Senna


In [361]:
AS_RS = pd.DataFrame(AS_R.groupby( ['grid'] ).status.value_counts()).rename( columns = {'status':'count' } ).reset_index()
AS_RS.status.unique()

array(['Did not qualify', 'Finished', 'Collision', 'Spun off', 'Accident',
       'Engine', 'Disqualified', 'Electrical', 'Out of fuel', '+2 Laps',
       'Radiator', 'Wheel', 'Wheel bearing', '+1 Lap', '+5 Laps',
       'Differential', 'Alternator', 'Gearbox', 'Transmission', 'Tyre',
       'Fuel system', 'Hydraulics', 'Throttle', 'CV joint', 'Clutch',
       'Oil pressure', '+3 Laps', 'Turbo'], dtype=object)

In [362]:
AS_RS

,grid,status,count
0,0,Did not qualify,1
1,1,Finished,40
2,1,Collision,5
3,1,Spun off,4
4,1,Accident,3
...,...,...,...
58,13,Finished,1
59,13,Turbo,1
60,14,Finished,1
61,16,Turbo,1


In [363]:
AS_finished = AS_R[ AS_R.status.isin(['Finished', '+1 Lap' ,'+2 Laps', '+3 Laps', '+4 Laps', '+5 Laps']) ]
AS_finished

,raceId,driverId,number,grid,final_pos,laps,milliseconds,status,standing_score,standing_pos,wins,forename,surname
3,273,102,8,2,2,72,6004906,Finished,6.0,2.0,0.0,Ayrton,Senna
4,274,102,8,3,1,71,6675485,Finished,16.0,1.0,1.0,Ayrton,Senna
5,275,102,8,4,1,76,6646570,Finished,26.0,1.0,2.0,Ayrton,Senna
7,277,102,8,3,2,65,5564558,Finished,32.0,2.0,2.0,Ayrton,Senna
8,278,102,8,3,1,78,6730947,Finished,42.0,1.0,3.0,Ayrton,Senna
...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,438,102,19,19,6,68,\N,+2 Laps,2.0,12.0,0.0,Ayrton,Senna
152,441,102,19,13,2,31,3675186,Finished,5.0,9.0,0.0,Ayrton,Senna
153,442,102,19,9,7,68,\N,+2 Laps,5.0,10.0,0.0,Ayrton,Senna
156,445,102,19,7,3,71,5431860,Finished,9.0,9.0,0.0,Ayrton,Senna


## Graphics

In [364]:
AS_FP = pd.DataFrame(AS_finished.groupby( ['grid', 'status'] ).final_pos.value_counts()).rename( columns = {'final_pos':'number' } ).reset_index()
AS_FP.status = AS_FP.status.astype(str).astype(Finished_type)
#AS_FP.final_pos = AS_FP.final_pos.astype(str).astype(Grid_type)
AS_FP.grid = AS_FP.grid.astype(str).astype(Grid_type)
#AS_FP.grid = AS_FP.grid.astype(str)
AS_FP = AS_FP.sort_values(by = [ 'final_pos', 'status' ])
AS_FP.head()

,grid,status,final_pos,number
1,1,Finished,1,29
8,2,Finished,1,5
16,3,Finished,1,5
20,4,Finished,1,1
25,5,Finished,1,1


In [365]:
print(AS_FP.nunique())
print(AS_FP.status.unique())

grid         11
status        5
final_pos     9
number        8
dtype: int64
['Finished', '+2 Laps', '+5 Laps', '+1 Lap', '+3 Laps']
Categories (16, object): ['Finished' < '+1 Lap' < '+2 Laps' < '+3 Laps' ... '+12 Laps' < '+13 Laps' <
                          '+14 Laps' < '+15 Laps']


In [366]:
AS_P1 = px.bar_polar(
    AS_FP[ AS_FP.status == 'Finished' ]
  , r = 'number'
  , theta = 'grid'
  , color = 'final_pos'
  , color_discrete_sequence = px.colors.sequential.Plasma_r
)

AS_P1.update_layout(
    title = format_title("Ayrton's results per qualy position", 'Colored by final position')
    , autosize=False
    , width=720
    , height=720
    , template = 'plotly_dark'
)
AS_P1.show()

In [367]:
AS_P2 = px.bar_polar(
    AS_FP[ AS_FP.status == '+1 Lap' ]
  , r = 'number'
  , theta = 'grid'
  , color = 'final_pos'
  , color_discrete_sequence = px.colors.sequential.Plasma_r
)

AS_P2.update_layout(
    title = format_title("Ayrton's results per qualy position", 'Colored by final position')
    , autosize=False
    , width=720
    , height=720
    , template = 'plotly_dark'
)
AS_P2.show()

In [368]:
AS_P3 = px.bar_polar(
    AS_FP[ AS_FP.status == '+2 Laps' ]
  , r = 'number'
  , theta = 'grid'
  , color = 'final_pos'
  , color_discrete_sequence = px.colors.sequential.Plasma_r
)

AS_P3.update_layout(
    title = format_title("Ayrton's results per qualy position", 'Colored by final position')
    , autosize=False
    , width=720
    , height=720
    , template = 'plotly_dark'
)
AS_P3.show()

In [369]:
AS_P4 = px.bar_polar(
    AS_FP[ AS_FP.status == '+3 Laps' ]
  , r = 'number'
  , theta = 'grid'
  , color = 'final_pos'
  , color_discrete_sequence = px.colors.sequential.Plasma_r
)

AS_P4.update_layout(
    title = format_title("Ayrton's results per qualy position", 'Colored by final position')
    , autosize=False
    , width=720
    , height=720
    , template = 'plotly_dark'
)
AS_P4.show()

In [370]:
AS_P5 = px.bar_polar(
    AS_FP[ AS_FP.status == '+5 Laps' ]
  , r = 'number'
  , theta = 'grid'
  , color = 'final_pos'
  , color_discrete_sequence = px.colors.sequential.Plasma_r
)

AS_P5.update_layout(
    title = format_title("Ayrton's results per qualy position", 'Colored by final position')
    , autosize=False
    , width=720
    , height=720
    , template = 'plotly_dark'
)
AS_P5.show()

#Who do we think the contenders are?

To determine the contenders, we reviewed the drivers' championship positions from 2021 to the current one, and took those drivers who were in the first 5 places. And we chose the 6 pilots with the highest recurrence

1.   Max Verstappen
2.   Sergio Pérez
3.   Charles Leclerc
4.   George Russell
5.   Lewis Hamilton
6.   Carlos Sainz



##Contenders

###Verstappen

In [371]:
MV = drivers[ (drivers.forename == 'Max') & (drivers.surname == 'Verstappen') ].reset_index().drop(columns='index')
MV_Id = MV.loc[0, 'driverId']
MV

,driverId,dorsal,code,forename,surname,dob,nationality
0,830,33,VER,Max,Verstappen,1997-09-30,Dutch


###Checo

In [372]:
SP = drivers[ (drivers.forename == 'Sergio') & (drivers.surname == 'Pérez') ].reset_index().drop(columns='index')
SP_Id = SP.loc[0, 'driverId']
SP

,driverId,dorsal,code,forename,surname,dob,nationality
0,815,11,PER,Sergio,Pérez,1990-01-26,Mexican


###Leclerc

In [373]:
CL = drivers[ (drivers.forename == 'Charles') & (drivers.surname == 'Leclerc') ].reset_index().drop(columns='index')
CL_Id = CL.loc[0, 'driverId']
CL

,driverId,dorsal,code,forename,surname,dob,nationality
0,844,16,LEC,Charles,Leclerc,1997-10-16,Monegasque


###Russell

In [374]:
GR = drivers[ (drivers.forename == 'George') & (drivers.surname == 'Russell') ].reset_index().drop(columns='index')
GR_Id = GR.loc[0, 'driverId']
GR

,driverId,dorsal,code,forename,surname,dob,nationality
0,847,63,RUS,George,Russell,1998-02-15,British


###Hamilton

In [375]:
LH = drivers[ (drivers.forename == 'Lewis') & (drivers.surname == 'Hamilton') ].reset_index().drop(columns='index')
LH_Id = LH.loc[0, 'driverId']
LH

,driverId,dorsal,code,forename,surname,dob,nationality
0,1,44,HAM,Lewis,Hamilton,1985-01-07,British


###Sainz

In [376]:
CS = drivers[ (drivers.forename == 'Carlos') & (drivers.surname == 'Sainz') ].reset_index().drop(columns='index')
CS_Id = CS.loc[0, 'driverId']
CS

,driverId,dorsal,code,forename,surname,dob,nationality
0,832,55,SAI,Carlos,Sainz,1994-09-01,Spanish


##Code

In [377]:
CC = pd.concat([MV, SP, CL, GR, LH, CS]).reset_index().drop(columns= 'index')
CC['full_name'] = CC.forename + " " + CC.surname
CC = CC.drop(columns = [ 'forename', 'surname' ])
CC

,driverId,dorsal,code,dob,nationality,full_name
0,830,33,VER,1997-09-30,Dutch,Max Verstappen
1,815,11,PER,1990-01-26,Mexican,Sergio Pérez
2,844,16,LEC,1997-10-16,Monegasque,Charles Leclerc
3,847,63,RUS,1998-02-15,British,George Russell
4,1,44,HAM,1985-01-07,British,Lewis Hamilton
5,832,55,SAI,1994-09-01,Spanish,Carlos Sainz


In [378]:
MV_s = driver_standings[ driver_standings.driverId == MV_Id ].merge( CC, on = 'driverId', how = 'left' ).drop( columns = ['dob', 'nationality', 'dorsal', 'code' ] )
SP_s = driver_standings[ driver_standings.driverId == SP_Id ].merge( CC, on = 'driverId', how = 'left' ).drop( columns = ['dob', 'nationality', 'dorsal', 'code' ] )
CL_s = driver_standings[ driver_standings.driverId == CL_Id ].merge( CC, on = 'driverId', how = 'left' ).drop( columns = ['dob', 'nationality', 'dorsal', 'code' ] )
GR_s = driver_standings[ driver_standings.driverId == GR_Id ].merge( CC, on = 'driverId', how = 'left' ).drop( columns = ['dob', 'nationality', 'dorsal', 'code' ] )
LH_s = driver_standings[ driver_standings.driverId == LH_Id ].merge( CC, on = 'driverId', how = 'left' ).drop( columns = ['dob', 'nationality', 'dorsal', 'code' ] )
CS_s = driver_standings[ driver_standings.driverId == CS_Id ].merge( CC, on = 'driverId', how = 'left' ).drop( columns = ['dob', 'nationality', 'dorsal', 'code' ] )
CC_standings = pd.concat([ MV_s, SP_s, CL_s, GR_s, LH_s, CS_s ]).reset_index().drop(columns= 'index')
#CC_standings = CC_standing.merge( CC, on = 'driverId', how = 'left' ).drop( columns = ['dob', 'nationality', 'driverId', 'dorsal', 'code' ] ).drop(columns = 'index')
CC_standings

,raceId,driverId,standing_score,standing_pos,wins,full_name
0,969,830,10.0,5,0,Max Verstappen
1,926,830,0.0,13,0,Max Verstappen
2,927,830,6.0,10,0,Max Verstappen
3,928,830,6.0,11,0,Max Verstappen
4,929,830,6.0,11,0,Max Verstappen
...,...,...,...,...,...,...
1127,1106,832,68.0,5,0,Carlos Sainz
1128,1107,832,82.0,5,0,Carlos Sainz
1129,1108,832,83.0,5,0,Carlos Sainz
1130,1109,832,87.0,6,0,Carlos Sainz


In [379]:
CC_standings.driverId.unique()

array([830, 815, 844, 847,   1, 832])

In [382]:
MV_r = R[ R.driverId == MV_Id ].drop(columns = 'driverId').merge(MV_s, on = 'raceId', how = 'right')
SP_r = R[ R.driverId == SP_Id ].drop(columns = 'driverId').merge(SP_s, on = 'raceId', how = 'right')
CL_r = R[ R.driverId == CL_Id ].drop(columns = 'driverId').merge(CL_s, on = 'raceId', how = 'right')
GR_r = R[ R.driverId == GR_Id ].drop(columns = 'driverId').merge(GR_s, on = 'raceId', how = 'right')
LH_r = R[ R.driverId == LH_Id ].drop(columns = 'driverId').merge(LH_s, on = 'raceId', how = 'right')
CS_r = R[ R.driverId == CS_Id ].drop(columns = 'driverId').merge(CS_s, on = 'raceId', how = 'right')
CC_result = pd.concat([ MV_r, SP_r, CL_r, GR_r, LH_r, CS_r ]).reset_index().drop(columns ='index')
CC_result
#CL_r

,raceId,number,grid,final_pos,laps,milliseconds,status,driverId,standing_score,standing_pos,wins,full_name
0,969,33,5.0,5,57.0,5080499,Finished,830,10.0,5,0,Max Verstappen
1,926,33,11.0,\N,32.0,\N,Engine,830,0.0,13,0,Max Verstappen
2,927,33,6.0,7,56.0,6163555,Finished,830,6.0,10,0,Max Verstappen
3,928,33,13.0,17,52.0,\N,Transmission,830,6.0,11,0,Max Verstappen
4,929,33,15.0,\N,34.0,\N,Electrical,830,6.0,11,0,Max Verstappen
...,...,...,...,...,...,...,...,...,...,...,...,...
1127,1106,55,11.0,5,70.0,5659888,Finished,832,68.0,5,0,Carlos Sainz
1128,1107,55,3.0,6,71.0,5164984,Finished,832,82.0,5,0,Carlos Sainz
1129,1108,55,5.0,10,52.0,5136386,Finished,832,83.0,5,0,Carlos Sainz
1130,1109,55,11.0,8,70.0,5959707,Finished,832,87.0,6,0,Carlos Sainz


In [383]:
CC_results = CC_result.copy()
CC_results.full_name.unique()

array(['Max Verstappen', 'Sergio Pérez', 'Charles Leclerc',
       'George Russell', 'Lewis Hamilton', 'Carlos Sainz'], dtype=object)

##Results

In [464]:
CC_R = CC_results.copy()
CC_R.driverId.unique()

array([830, 815, 844, 847,   1, 832])

In [465]:
CC_R

,raceId,number,grid,final_pos,laps,milliseconds,status,driverId,standing_score,standing_pos,wins,full_name
0,969,33,5.0,5,57.0,5080499,Finished,830,10.0,5,0,Max Verstappen
1,926,33,11.0,\N,32.0,\N,Engine,830,0.0,13,0,Max Verstappen
2,927,33,6.0,7,56.0,6163555,Finished,830,6.0,10,0,Max Verstappen
3,928,33,13.0,17,52.0,\N,Transmission,830,6.0,11,0,Max Verstappen
4,929,33,15.0,\N,34.0,\N,Electrical,830,6.0,11,0,Max Verstappen
...,...,...,...,...,...,...,...,...,...,...,...,...
1127,1106,55,11.0,5,70.0,5659888,Finished,832,68.0,5,0,Carlos Sainz
1128,1107,55,3.0,6,71.0,5164984,Finished,832,82.0,5,0,Carlos Sainz
1129,1108,55,5.0,10,52.0,5136386,Finished,832,83.0,5,0,Carlos Sainz
1130,1109,55,11.0,8,70.0,5959707,Finished,832,87.0,6,0,Carlos Sainz


In [387]:
CC_finished = CC_R[ CC_R.status.isin(['Finished', '+1 Lap' ,'+2 Laps', '+3 Laps', '+4 Laps', '+5 Laps']) ]
CC_finished

,raceId,number,grid,final_pos,laps,milliseconds,status,driverId,standing_score,standing_pos,wins,full_name
0,969,33,5.0,5,57.0,5080499,Finished,830,10.0,5,0,Max Verstappen
2,927,33,6.0,7,56.0,6163555,Finished,830,6.0,10,0,Max Verstappen
5,930,33,6.0,11,65.0,\N,+1 Lap,830,6.0,11,0,Max Verstappen
7,932,33,19.0,15,69.0,\N,+1 Lap,830,6.0,14,0,Max Verstappen
8,933,33,7.0,8,70.0,\N,+1 Lap,830,10.0,14,0,Max Verstappen
...,...,...,...,...,...,...,...,...,...,...,...,...
1126,1105,55,2.0,5,66.0,5323638,Finished,832,58.0,6,0,Carlos Sainz
1127,1106,55,11.0,5,70.0,5659888,Finished,832,68.0,5,0,Carlos Sainz
1128,1107,55,3.0,6,71.0,5164984,Finished,832,82.0,5,0,Carlos Sainz
1129,1108,55,5.0,10,52.0,5136386,Finished,832,83.0,5,0,Carlos Sainz


In [398]:
CC_finished['grid'] = CC_finished.grid.astype(int)
CC_finished

<ipython-input-398-0adc98f923d5>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,raceId,number,grid,final_pos,laps,milliseconds,status,driverId,standing_score,standing_pos,wins,full_name
0,969,33,5,5,57.0,5080499,Finished,830,10.0,5,0,Max Verstappen
2,927,33,6,7,56.0,6163555,Finished,830,6.0,10,0,Max Verstappen
5,930,33,6,11,65.0,\N,+1 Lap,830,6.0,11,0,Max Verstappen
7,932,33,19,15,69.0,\N,+1 Lap,830,6.0,14,0,Max Verstappen
8,933,33,7,8,70.0,\N,+1 Lap,830,10.0,14,0,Max Verstappen
...,...,...,...,...,...,...,...,...,...,...,...,...
1126,1105,55,2,5,66.0,5323638,Finished,832,58.0,6,0,Carlos Sainz
1127,1106,55,11,5,70.0,5659888,Finished,832,68.0,5,0,Carlos Sainz
1128,1107,55,3,6,71.0,5164984,Finished,832,82.0,5,0,Carlos Sainz
1129,1108,55,5,10,52.0,5136386,Finished,832,83.0,5,0,Carlos Sainz


##Graphics

##Per contender

In [462]:
CC_FP = pd.DataFrame(CC_finished.groupby( ['grid', 'full_name'] ).final_pos.value_counts()).rename( columns = {'final_pos':'races' } ).reset_index()
CC_FP.grid = CC_FP.grid.astype(int)#.astype(str)#.astype(Grid_type)
CC_FP.final_pos = CC_FP.final_pos.astype(Grid_type)
CC_FP = CC_FP.sort_values(by  = ['final_pos'])
CC_FP

,grid,full_name,final_pos,races
43,2,Lewis Hamilton,1,27
119,4,Max Verstappen,1,4
28,1,Sergio Pérez,1,1
26,1,Max Verstappen,1,22
116,4,Lewis Hamilton,1,3
...,...,...,...,...
356,12,George Russell,18,1
148,5,Lewis Hamilton,18,1
495,20,George Russell,19,1
477,19,Charles Leclerc,19,1


In [463]:
CC_FP.grid.unique()

array([ 2,  4,  1, 14,  3, 10,  6,  5,  7,  9, 18,  8, 12, 15, 20, 17, 16,
       21, 22, 11, 19,  0, 13, 24, 23])

In [450]:
GR_P = px.bar_polar(
    CC_FP[ CC_FP.full_name == 'George Russell' ]
  , r = 'races'
  , theta = 'final_pos'
  , color = 'grid'
  , color_continuous_scale = px.colors.sequential.Plasma_r
)

GR_P.update_layout(
    title = format_title("Russell's results per final position", 'Colored by qualy position')
    , autosize=False
    , width=720
    , height=720
    , template = 'plotly_dark'
)
GR_P.show()

In [445]:
CS_P = px.bar_polar(
    CC_FP[CC_FP.full_name == 'Carlos Sainz'].sort_values(by =['final_pos'])
  , r = 'races'
  , theta = 'final_pos'
  , color = 'grid'
  , color_continuous_scale = px.colors.sequential.Plasma_r
)
CS_P.update_layout(
    title = format_title("Sainz's results per final position", 'Colored by qualy position')
    , autosize=False
    , width=720
    , height=720
    , template = 'plotly_dark'
)
CS_P.show()

In [447]:
SP_P = px.bar_polar(
    CC_FP[ CC_FP.full_name == 'Sergio Pérez' ]
  , r = 'races'
  , theta = 'final_pos'
  , color = 'grid'
  , color_continuous_scale = px.colors.sequential.Plasma_r
)

SP_P.update_layout(
    title = format_title("Checos's results per final position", 'Colored by qualy position')
    , autosize=False
    , width=720
    , height=720
    , template = 'plotly_dark'
)
SP_P.show()

In [451]:
CL_P = px.bar_polar(
    CC_FP[ CC_FP.full_name == 'Charles Leclerc' ]
  , r = 'races'
  , theta = 'final_pos'
  , color = 'grid'
  , color_continuous_scale = px.colors.sequential.Plasma_r
)

CL_P.update_layout(
    title = format_title("Leclerc's results per final position", 'Colored by qualy position')
    , autosize=False
    , width=720
    , height=720
    , template = 'plotly_dark'
)
CL_P.show()

In [449]:
LH_P = px.bar_polar(
    CC_FP[ CC_FP.full_name == 'Lewis Hamilton' ]
  , r = 'races'
  , theta = 'final_pos'
  , color = 'grid'
  , color_continuous_scale = px.colors.sequential.Plasma_r
)

LH_P.update_layout(
    title = format_title("Hamilton's results per final position", 'Colored by qualy position')
    , autosize=False
    , width=720
    , height=720
    , template = 'plotly_dark'
)
LH_P.show()

In [448]:
MV_P = px.bar_polar(
    CC_FP[ CC_FP.full_name == 'Max Verstappen' ]
  , r = 'races'
  , theta = 'final_pos'
  , color = 'grid'
  , color_continuous_scale = px.colors.sequential.Plasma_r
)
MV_P.update_layout(
    title = format_title("Verstappen's results per final position", 'Colored by qualy position')
    , autosize=False
    , width=720
    , height=720
    , template = 'plotly_dark'
)
MV_P.show()

##General

In [467]:
CC_R['Finishing'] = 'No'
CC_R.loc[ CC_R.status.isin(['Finished', '+1 Lap' ,'+2 Laps', '+3 Laps', '+4 Laps', '+5 Laps']), 'Finishing' ] = 'Yes'
CC_RS = pd.DataFrame(CC_R.groupby( ['full_name'] ).Finishing.value_counts()).rename( columns = {'Finishing':'count' } ).reset_index()
CC_aux = pd.DataFrame(CC_R.groupby( [ 'full_name'] ).Finishing.value_counts(normalize=True)*100).rename(columns={'Finishing':'normalize'})
CC_RSN = CC_RS.merge(CC_aux, on = [ 'full_name', 'Finishing' ], how = 'left' )
CC_RSN

,full_name,Finishing,count,normalize
0,Carlos Sainz,Yes,140,80.000000
1,Carlos Sainz,No,35,20.000000
2,Charles Leclerc,Yes,95,82.608696
3,Charles Leclerc,No,20,17.391304
4,George Russell,Yes,79,84.042553
5,George Russell,No,15,15.957447
6,Lewis Hamilton,Yes,292,90.683230
7,Lewis Hamilton,No,30,9.316770
8,Max Verstappen,Yes,144,82.285714
9,Max Verstappen,No,31,17.714286


In [474]:
CC_GP =  make_subplots(specs = [ [ { 'secondary_y':True } ] ] )
for driver in CC_RSN.full_name.unique():
  CC_GP.add_trace(go.Bar( x = CC_RSN[ CC_RSN.full_name == driver ].Finishing, y = CC_RSN[ CC_RSN.full_name == driver ]['count'], name = str(driver) ))
CC_GP.show()